In [5]:
import numpy as np
import matplotlib.pyplot as plt

# Import Qiskit.
from qiskit import BasicAer, IBMQ
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute, compile
from qiskit.tools.visualization import plot_histogram
from qiskit import Aer, execute
from qiskit.circuit import Gate
from qiskit.aqua.algorithms import Grover
from math import sqrt
from qiskit.extensions.unitary import UnitaryGate

In [6]:
def make_init_state(memory, q, circ): 
    """Code the classical memory in a quantum state.
       Undefined states are written as superposition."""
    
    for i, element in enumerate(memory.flatten()):
        if element == 1:
            circ.x(q[i])
        elif element == 2:
            circ.h(q[i])
    return

def mcz(self, qubits):
    """Apply Multi-controlled Z gate.
    
    Args:
        qubits (list): list of qubits. The first n-1 are control
                       qubits, the nth is the target qubit.
    """
    if isinstance(qubits, QuantumRegister):
        # Convert register to list of qubits in register
        qubits = qubits[:]
    num_qubits = len(qubits)
    return self.append(Gate("mcz", num_qubits, []), qargs=qubits)

# Monkey patching.
QuantumCircuit.mcz = mcz

def put_oracle(circ, q):
    """Append the oracle to the circuit."""
    # Put all winning combinations in a hard-coded 3x3 window.
    # Cols.
    circ.mcz([q[9], q[0], q[3], q[6]])
    circ.mcz([q[10], q[1], q[4], q[7]])
    circ.mcz([q[11], q[2], q[5], q[8]])
    # Rows.
    circ.mcz([q[9], q[0], q[1], q[2]])
    circ.mcz([q[10], q[3], q[4], q[5]])
    circ.mcz([q[11], q[6], q[7], q[8]])
    # Diagonals.
    circ.mcz([q[12], q[0], q[4], q[8]])
    circ.mcz([q[12], q[2], q[4],q[6]])
    return    
    
def make_inversion_about_average(circ, q):
    """Make inversion about the average, the second step of Grover's algorithm."""
    # Hadamards everywhere
    for i in range(9):
        circ.h(q[i])
    # D matrix: flips the sign of the state |000> only
    for i in range(9):
        circ.x(q[i])
    # Multiple control z gate
    circ.mcz([q[0], q[1], q[2], q[3], q[4], q[5], q[6], q[7], q[8]])
    for i in range(9):
        circ.x(q[i])
    for i in range(9):
        circ.h(q[i])
    return

In [7]:
def build_circuit(wcols, wrows, n_ancillas, n_grover_iters, memory):
    """Make a quantum circuit with an initial state and Grover steps `n_grover_iters` times."""
    # Make registers and initialize circuit.
    nqubits = wcols*wrows + n_ancillas
    q = QuantumRegister(nqubits, 'q')
    c = ClassicalRegister(nqubits - n_ancillas, 'c')
    circ = QuantumCircuit(q, c)

    # Initialize qubits with memory.
    make_init_state(memory, q, circ)
    # Initialize ancillas.
    circ.h(q[-n_ancillas:])
    circ.barrier()
    
    # Grover's search.
    for _ in range(n_grover_iters):
        put_oracle(circ, q)
        make_inversion_about_average(circ, q)
        
    # Measure qubits.
    circ.measure(q[:-n_ancillas], c)
    return circ

def qasm_simulate_circuit(circ, shots=1024):
    backend = Aer.get_backend('qasm_simulator')
    job = execute(circ, backend, basis_gates=['mcz','h','U','x'], shots=shots)
    result = job.result()
    counts = result.get_counts()
    print(f"Max counts: {max(counts.values())}")
    plot_histogram(counts) 
    return counts

def filter_counts(counts, shots):
    threshold = max(counts.values())/4
    n_wanted_results = 10
    
    keys = np.fromiter(counts.keys(), dtype=np.int)
    vals = np.fromiter(counts.values(), dtype=np.int)
    
    # Threshold.
    #keys = keys[vals > threshold]
    #vals = vals[vals > threshold]
    keys = keys[np.argsort(vals)][-n_wanted_results:]
    vals = vals[np.argsort(vals)][-n_wanted_results:]
    
    for i in range(len(keys)):
        key = f"{keys[i]:>09}"
        print(f"probability: {100*vals[i]/shots:4.2f} %")
        print(f"{key[8:5:-1]}\n{key[5:2:-1]}\n{key[2:0:-1]}{key[0]}\n")
    
    return

# Parameters and simulate circuit.

We simulate the Grover's search algorithm maximizing the winning moves that make 3 in a row in a 3x3 slinding window. We plot the most probable results and the 3x3 window state.

In [8]:
wcols = 3
wrows = 3
n_ancillas = 4
n_grover_iters = 1
shots = 2048

memory = np.array([[2, 2, 2], 
                   [2, 2, 0],
                   [1, 0, 0]])

circ = build_circuit(wcols, wrows, n_ancillas, n_grover_iters, memory)
counts = qasm_simulate_circuit(circ, shots=shots)
#plot_histogram(counts)
filter_counts(counts, shots)

Max counts: 78
probability: 3.08 %
100
100
100

probability: 3.12 %
111
000
100

probability: 3.12 %
110
000
100

probability: 3.17 %
111
010
100

probability: 3.22 %
110
110
100

probability: 3.37 %
100
010
100

probability: 3.37 %
101
110
100

probability: 3.47 %
011
110
100

probability: 3.71 %
001
010
100

probability: 3.81 %
110
100
100

